# 融合检索：结合向量搜索和关键词搜索 (Fusion Retrieval: Combining Vector and Keyword Search)

在这个笔记本中，我实现了一个融合检索系统，该系统结合了语义向量搜索和基于关键词的 BM25 检索的优势。这种方法通过捕获概念相似性和精确关键词匹配来提高检索质量。

## 为什么融合检索很重要

传统的 RAG 系统通常仅依赖向量搜索，但这有其局限性：

- 向量搜索在语义相似性方面表现出色，但可能错过精确的关键词匹配
- 关键词搜索对特定术语很有效，但缺乏语义理解
- 不同的查询在不同的检索方法下表现更好

融合检索通过以下方式为我们提供了两全其美的解决方案：

- 同时执行基于向量和基于关键词的检索
- 对每种方法的分数进行标准化
- 使用加权公式将它们结合起来
- 基于组合分数对文档进行排名

## 设置环境
我们首先导入必要的库。

In [1]:
import os
import numpy as np
from rank_bm25 import BM25Okapi
import fitz
from openai import OpenAI
import re
import json
import time
from sklearn.metrics.pairwise import cosine_similarity

## 设置 OpenAI API 客户端
我们初始化 OpenAI 客户端来生成嵌入向量和响应。

In [ ]:
# 使用基础 URL 和 API 密钥初始化 OpenAI 客户端
client = OpenAI(
    base_url="https://api.studio.nebius.com/v1/",
    api_key=os.getenv("OPENAI_API_KEY")  # 从环境变量中获取 API 密钥
)

## 文档处理函数

In [3]:
def extract_text_from_pdf(pdf_path):
    """
    从 PDF 文件中提取文本内容。
    
    参数:
        pdf_path (str): PDF 文件的路径
        
    返回:
        str: 提取的文本内容
    """
    print(f"从 {pdf_path} 提取文本...")  # 打印正在处理的 PDF 路径
    pdf_document = fitz.open(pdf_path)  # 使用 PyMuPDF 打开 PDF 文件
    text = ""  # 初始化一个空字符串来存储提取的文本
    
    # 遍历 PDF 中的每一页
    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]  # 获取页面对象
        text += page.get_text()  # 从页面提取文本并追加到文本字符串中
    
    return text  # 返回提取的文本内容

In [4]:
def chunk_text(text, chunk_size=1000, chunk_overlap=200):
    """
    将文本分割成重叠的块。
    
    参数:
        text (str): 要分块的输入文本
        chunk_size (int): 每个块的字符数大小
        chunk_overlap (int): 块之间的重叠字符数
        
    返回:
        List[Dict]: 包含文本和元数据的块列表
    """
    chunks = []  # 初始化一个空列表来存储块
    
    # 按指定的块大小和重叠遍历文本
    for i in range(0, len(text), chunk_size - chunk_overlap):
        chunk = text[i:i + chunk_size]  # 提取指定大小的块
        if chunk:  # 确保我们不添加空块
            chunk_data = {
                "text": chunk,  # 块文本
                "metadata": {
                    "start_char": i,  # 块的起始字符索引
                    "end_char": i + len(chunk)  # 块的结束字符索引
                }
            }
            chunks.append(chunk_data)  # 将块数据添加到列表中
    
    print(f"创建了 {len(chunks)} 个文本块")  # 打印创建的块数量
    return chunks  # 返回块列表

In [5]:
def clean_text(text):
    """
    通过移除多余的空白和特殊字符来清理文本。
    
    参数:
        text (str): 输入文本
        
    返回:
        str: 清理后的文本
    """
    # 将多个空白字符（包括换行符和制表符）替换为单个空格
    text = re.sub(r'\s+', ' ', text)
    
    # 通过将制表符和换行符替换为空格来修复常见的 OCR 问题
    text = text.replace('\\t', ' ')
    text = text.replace('\\n', ' ')
    
    # 移除任何前导或尾随空白，并确保单词之间只有单个空格
    text = ' '.join(text.split())
    
    return text

## 创建我们的向量存储

In [6]:
def create_embeddings(texts, model="BAAI/bge-en-icl"):
    """
    为给定的文本创建嵌入向量。
    
    参数:
        texts (str or List[str]): 输入文本
        model (str): 嵌入模型名称
        
    返回:
        List[List[float]]: 嵌入向量
    """
    # 处理字符串和列表输入
    input_texts = texts if isinstance(texts, list) else [texts]
    
    # 如果需要，分批处理（OpenAI API 限制）
    batch_size = 100
    all_embeddings = []
    
    # 分批遍历输入文本
    for i in range(0, len(input_texts), batch_size):
        batch = input_texts[i:i + batch_size]  # 获取当前批次的文本
        
        # 为当前批次创建嵌入向量
        response = client.embeddings.create(
            model=model,
            input=batch
        )
        
        # 从响应中提取嵌入向量
        batch_embeddings = [item.embedding for item in response.data]
        all_embeddings.extend(batch_embeddings)  # 将批次嵌入向量添加到列表中
    
    # 如果输入是字符串，只返回第一个嵌入向量
    if isinstance(texts, str):
        return all_embeddings[0]
    
    # 否则返回所有嵌入向量
    return all_embeddings

In [7]:
class SimpleVectorStore:
    """
    使用 NumPy 的简单向量存储实现。
    """
    def __init__(self):
        self.vectors = []  # 存储嵌入向量的列表
        self.texts = []  # 存储文本内容的列表
        self.metadata = []  # 存储元数据的列表
    
    def add_item(self, text, embedding, metadata=None):
        """
        向向量存储中添加一个项目。
        
        参数:
            text (str): 文本内容
            embedding (List[float]): 嵌入向量
            metadata (Dict, optional): 附加元数据
        """
        self.vectors.append(np.array(embedding))  # 追加嵌入向量
        self.texts.append(text)  # 追加文本内容
        self.metadata.append(metadata or {})  # 追加元数据（如果为 None 则为空字典）
    
    def add_items(self, items, embeddings):
        """
        向向量存储中添加多个项目。
        
        参数:
            items (List[Dict]): 文本项目列表
            embeddings (List[List[float]]): 嵌入向量列表
        """
        for i, (item, embedding) in enumerate(zip(items, embeddings)):
            self.add_item(
                text=item["text"],  # 从项目中提取文本
                embedding=embedding,  # 使用对应的嵌入向量
                metadata={**item.get("metadata", {}), "index": i}  # 合并项目元数据和索引
            )
    
    def similarity_search_with_scores(self, query_embedding, k=5):
        """
        查找与查询嵌入向量最相似的项目并返回相似度分数。
        
        参数:
            query_embedding (List[float]): 查询嵌入向量
            k (int): 返回结果的数量
            
        返回:
            List[Tuple[Dict, float]]: 前 k 个最相似的项目及其分数
        """
        if not self.vectors:
            return []  # 如果没有存储向量则返回空列表
        
        # 将查询嵌入向量转换为 numpy 数组
        query_vector = np.array(query_embedding)
        
        # 使用余弦相似度计算相似性
        similarities = []
        for i, vector in enumerate(self.vectors):
            similarity = cosine_similarity([query_vector], [vector])[0][0]  # 计算余弦相似度
            similarities.append((i, similarity))  # 追加索引和相似度分数
        
        # 按相似度降序排序
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # 返回前 k 个结果及其分数
        results = []
        for i in range(min(k, len(similarities))):
            idx, score = similarities[i]
            results.append({
                "text": self.texts[idx],  # 按索引检索文本
                "metadata": self.metadata[idx],  # 按索引检索元数据
                "similarity": float(score)  # 添加相似度分数
            })
        
        return results
    
    def get_all_documents(self):
        """
        获取存储中的所有文档。
        
        返回:
            List[Dict]: 所有文档
        """
        return [{"text": text, "metadata": meta} for text, meta in zip(self.texts, self.metadata)]  # 合并文本和元数据

## BM25 实现

In [8]:
def create_bm25_index(chunks):
    """
    从给定的块创建 BM25 索引。
    
    参数:
        chunks (List[Dict]): 文本块列表
        
    返回:
        BM25Okapi: BM25 索引
    """
    # 从每个块中提取文本
    texts = [chunk["text"] for chunk in chunks]
    
    # 通过按空白分割对每个文档进行分词
    tokenized_docs = [text.split() for text in texts]
    
    # 使用分词文档创建 BM25 索引
    bm25 = BM25Okapi(tokenized_docs)
    
    # 打印 BM25 索引中的文档数量
    print(f"创建了包含 {len(texts)} 个文档的 BM25 索引")
    
    return bm25

In [9]:
def bm25_search(bm25, chunks, query, k=5):
    """
    使用查询搜索 BM25 索引。
    
    参数:
        bm25 (BM25Okapi): BM25 索引
        chunks (List[Dict]): 文本块列表
        query (str): 查询字符串
        k (int): 返回结果的数量
        
    返回:
        List[Dict]: 前 k 个结果及其分数
    """
    # 通过将查询分割成单个词来对查询进行分词
    query_tokens = query.split()
    
    # 获取查询词对索引文档的 BM25 分数
    scores = bm25.get_scores(query_tokens)
    
    # 初始化一个空列表来存储结果及其分数
    results = []
    
    # 遍历分数和对应的块
    for i, score in enumerate(scores):
        # 创建元数据的副本以避免修改原始数据
        metadata = chunks[i].get("metadata", {}).copy()
        # 向元数据添加索引
        metadata["index"] = i
        
        results.append({
            "text": chunks[i]["text"],
            "metadata": metadata,  # 添加带索引的元数据
            "bm25_score": float(score)
        })
    
    # 按 BM25 分数降序排序结果
    results.sort(key=lambda x: x["bm25_score"], reverse=True)
    
    # 返回前 k 个结果
    return results[:k]

## 融合检索函数

In [10]:
def fusion_retrieval(query, chunks, vector_store, bm25_index, k=5, alpha=0.5):
    """
    执行结合基于向量和 BM25 搜索的融合检索。
    
    参数:
        query (str): 查询字符串
        chunks (List[Dict]): 原始文本块
        vector_store (SimpleVectorStore): 向量存储
        bm25_index (BM25Okapi): BM25 索引
        k (int): 返回结果的数量
        alpha (float): 向量分数的权重（0-1），其中 1-alpha 是 BM25 权重
        
    返回:
        List[Dict]: 基于组合分数的前 k 个结果
    """
    print(f"对查询执行融合检索：{query}")
    
    # 定义小的 epsilon 以避免除零
    epsilon = 1e-8
    
    # 获取向量搜索结果
    query_embedding = create_embeddings(query)  # 为查询创建嵌入向量
    vector_results = vector_store.similarity_search_with_scores(query_embedding, k=len(chunks))  # 执行向量搜索
    
    # 获取 BM25 搜索结果
    bm25_results = bm25_search(bm25_index, chunks, query, k=len(chunks))  # 执行 BM25 搜索
    
    # 创建字典将文档索引映射到分数
    vector_scores_dict = {result["metadata"]["index"]: result["similarity"] for result in vector_results}
    bm25_scores_dict = {result["metadata"]["index"]: result["bm25_score"] for result in bm25_results}
    
    # 确保所有文档都有两种方法的分数
    all_docs = vector_store.get_all_documents()
    combined_results = []
    
    for i, doc in enumerate(all_docs):
        vector_score = vector_scores_dict.get(i, 0.0)  # 获取向量分数，如果未找到则为 0
        bm25_score = bm25_scores_dict.get(i, 0.0)  # 获取 BM25 分数，如果未找到则为 0
        combined_results.append({
            "text": doc["text"],
            "metadata": doc["metadata"],
            "vector_score": vector_score,
            "bm25_score": bm25_score,
            "index": i
        })
    
    # 将分数提取为数组
    vector_scores = np.array([doc["vector_score"] for doc in combined_results])
    bm25_scores = np.array([doc["bm25_score"] for doc in combined_results])
    
    # 标准化分数
    norm_vector_scores = (vector_scores - np.min(vector_scores)) / (np.max(vector_scores) - np.min(vector_scores) + epsilon)
    norm_bm25_scores = (bm25_scores - np.min(bm25_scores)) / (np.max(bm25_scores) - np.min(bm25_scores) + epsilon)
    
    # 计算组合分数
    combined_scores = alpha * norm_vector_scores + (1 - alpha) * norm_bm25_scores
    
    # 将组合分数添加到结果中
    for i, score in enumerate(combined_scores):
        combined_results[i]["combined_score"] = float(score)
    
    # 按组合分数降序排序
    combined_results.sort(key=lambda x: x["combined_score"], reverse=True)
    
    # 返回前 k 个结果
    top_results = combined_results[:k]
    
    print(f"使用融合检索检索到 {len(top_results)} 个文档")
    return top_results

## 文档处理流水线

In [11]:
def process_document(pdf_path, chunk_size=1000, chunk_overlap=200):
    """
    为融合检索处理文档。
    
    参数:
        pdf_path (str): PDF 文件的路径
        chunk_size (int): 每个块的字符数大小
        chunk_overlap (int): 块之间的重叠字符数
        
    返回:
        Tuple[List[Dict], SimpleVectorStore, BM25Okapi]: 块、向量存储和 BM25 索引
    """
    # 从 PDF 文件提取文本
    text = extract_text_from_pdf(pdf_path)
    
    # 清理提取的文本以移除多余的空白和特殊字符
    cleaned_text = clean_text(text)
    
    # 将清理后的文本分割成重叠的块
    chunks = chunk_text(cleaned_text, chunk_size, chunk_overlap)
    
    # 从每个块中提取文本内容用于嵌入创建
    chunk_texts = [chunk["text"] for chunk in chunks]
    print("为块创建嵌入向量...")
    
    # 为块文本创建嵌入向量
    embeddings = create_embeddings(chunk_texts)
    
    # 初始化向量存储
    vector_store = SimpleVectorStore()
    
    # 将块及其嵌入向量添加到向量存储中
    vector_store.add_items(chunks, embeddings)
    print(f"向向量存储添加了 {len(chunks)} 个项目")
    
    # 从块创建 BM25 索引
    bm25_index = create_bm25_index(chunks)
    
    # 返回块、向量存储和 BM25 索引
    return chunks, vector_store, bm25_index

## 响应生成

In [12]:
def generate_response(query, context):
    """
    基于查询和上下文生成响应。
    
    参数:
        query (str): 用户查询
        context (str): 从检索文档中获得的上下文
        
    返回:
        str: 生成的响应
    """
    # 定义系统提示词来指导 AI 助手
    system_prompt = """你是一个有用的 AI 助手。基于提供的上下文回答用户的问题。
    如果上下文不包含完全回答问题的相关信息，请承认这一限制。"""

    # 使用上下文和查询格式化用户提示词
    user_prompt = f"""上下文：
    {context}

    问题：{query}

    请基于提供的上下文回答问题。"""

    # 使用 OpenAI API 生成响应
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",  # 指定要使用的模型
        messages=[
            {"role": "system", "content": system_prompt},  # 指导助手的系统消息
            {"role": "user", "content": user_prompt}  # 包含上下文和查询的用户消息
        ],
        temperature=0.1  # 设置响应生成的温度
    )
    
    # 返回生成的响应
    return response.choices[0].message.content

## 主要检索函数

In [13]:
def answer_with_fusion_rag(query, chunks, vector_store, bm25_index, k=5, alpha=0.5):
    """
    使用融合 RAG 回答查询。
    
    参数:
        query (str): 用户查询
        chunks (List[Dict]): 文本块
        vector_store (SimpleVectorStore): 向量存储
        bm25_index (BM25Okapi): BM25 索引
        k (int): 要检索的文档数量
        alpha (float): 向量分数的权重
        
    返回:
        Dict: 查询结果，包括检索的文档和响应
    """
    # 使用融合检索方法检索文档
    retrieved_docs = fusion_retrieval(query, chunks, vector_store, bm25_index, k=k, alpha=alpha)
    
    # 通过用分隔符连接检索文档的文本来格式化上下文
    context = "\n\n---\n\n".join([doc["text"] for doc in retrieved_docs])
    
    # 基于查询和格式化的上下文生成响应
    response = generate_response(query, context)
    
    # 返回查询、检索的文档和生成的响应
    return {
        "query": query,
        "retrieved_documents": retrieved_docs,
        "response": response
    }

## 比较检索方法

In [14]:
def vector_only_rag(query, vector_store, k=5):
    """
    仅使用基于向量的 RAG 回答查询。
    
    参数:
        query (str): 用户查询
        vector_store (SimpleVectorStore): 向量存储
        k (int): 要检索的文档数量
        
    返回:
        Dict: 查询结果
    """
    # 创建查询嵌入向量
    query_embedding = create_embeddings(query)
    
    # 使用基于向量的相似性搜索检索文档
    retrieved_docs = vector_store.similarity_search_with_scores(query_embedding, k=k)
    
    # 通过用分隔符连接检索文档的文本来格式化上下文
    context = "\n\n---\n\n".join([doc["text"] for doc in retrieved_docs])
    
    # 基于查询和格式化的上下文生成响应
    response = generate_response(query, context)
    
    # 返回查询、检索的文档和生成的响应
    return {
        "query": query,
        "retrieved_documents": retrieved_docs,
        "response": response
    }

In [15]:
def bm25_only_rag(query, chunks, bm25_index, k=5):
    """
    仅使用基于 BM25 的 RAG 回答查询。
    
    参数:
        query (str): 用户查询
        chunks (List[Dict]): 文本块
        bm25_index (BM25Okapi): BM25 索引
        k (int): 要检索的文档数量
        
    返回:
        Dict: 查询结果
    """
    # 使用 BM25 搜索检索文档
    retrieved_docs = bm25_search(bm25_index, chunks, query, k=k)
    
    # 通过用分隔符连接检索文档的文本来格式化上下文
    context = "\n\n---\n\n".join([doc["text"] for doc in retrieved_docs])
    
    # 基于查询和格式化的上下文生成响应
    response = generate_response(query, context)
    
    # 返回查询、检索的文档和生成的响应
    return {
        "query": query,
        "retrieved_documents": retrieved_docs,
        "response": response
    }

## 评估函数

In [16]:
def compare_retrieval_methods(query, chunks, vector_store, bm25_index, k=5, alpha=0.5, reference_answer=None):
    """
    比较查询的不同检索方法。
    
    参数:
        query (str): 用户查询
        chunks (List[Dict]): 文本块
        vector_store (SimpleVectorStore): 向量存储
        bm25_index (BM25Okapi): BM25 索引
        k (int): 要检索的文档数量
        alpha (float): 融合检索中向量分数的权重
        reference_answer (str, optional): 用于比较的参考答案
        
    返回:
        Dict: 比较结果
    """
    print(f"\n=== 比较查询的检索方法：{query} ===\n")
    
    # 运行仅向量 RAG
    print("\n运行仅向量 RAG...")
    vector_result = vector_only_rag(query, vector_store, k)
    
    # 运行仅 BM25 RAG
    print("\n运行仅 BM25 RAG...")
    bm25_result = bm25_only_rag(query, chunks, bm25_index, k)
    
    # 运行融合 RAG
    print("\n运行融合 RAG...")
    fusion_result = answer_with_fusion_rag(query, chunks, vector_store, bm25_index, k, alpha)
    
    # 比较不同检索方法的响应
    print("\n比较响应...")
    comparison = evaluate_responses(
        query, 
        vector_result["response"], 
        bm25_result["response"], 
        fusion_result["response"],
        reference_answer
    )
    
    # 返回比较结果
    return {
        "query": query,
        "vector_result": vector_result,
        "bm25_result": bm25_result,
        "fusion_result": fusion_result,
        "comparison": comparison
    }

In [17]:
def evaluate_responses(query, vector_response, bm25_response, fusion_response, reference_answer=None):
    """
    评估不同检索方法的响应。
    
    参数:
        query (str): 用户查询
        vector_response (str): 仅向量 RAG 的响应
        bm25_response (str): 仅 BM25 RAG 的响应
        fusion_response (str): 融合 RAG 的响应
        reference_answer (str, optional): 参考答案
        
    返回:
        str: 响应评估
    """
    # 评估员的系统提示词，指导评估过程
    system_prompt = """你是 RAG 系统的专家评估员。比较三种不同检索方法的响应：
    1. 基于向量的检索：使用语义相似性进行文档检索
    2. BM25 关键词检索：使用关键词匹配进行文档检索
    3. 融合检索：结合向量和关键词方法

    基于以下标准评估响应：
    - 与查询的相关性
    - 事实正确性
    - 全面性
    - 清晰度和连贯性"""

    # 包含查询和响应的用户提示词
    user_prompt = f"""查询：{query}

    基于向量的响应：
    {vector_response}

    BM25 关键词响应：
    {bm25_response}

    融合响应：
    {fusion_response}
    """

    # 如果提供了参考答案，将其添加到提示词中
    if reference_answer:
        user_prompt += f"""
            参考答案：
            {reference_answer}
        """

    # 向用户提示词添加详细比较的指令
    user_prompt += """
    请提供这三个响应的详细比较。哪种方法在这个查询上表现最好，为什么？
    请具体说明每种方法在这个特定查询上的优缺点。
    """

    # 使用 meta-llama/Llama-3.2-3B-Instruct 生成评估
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",  # 指定要使用的模型
        messages=[
            {"role": "system", "content": system_prompt},  # 指导评估员的系统消息
            {"role": "user", "content": user_prompt}  # 包含查询和响应的用户消息
        ],
        temperature=0  # 设置响应生成的温度
    )
    
    # 返回生成的评估内容
    return response.choices[0].message.content

## 完整评估流水线

In [18]:
def evaluate_fusion_retrieval(pdf_path, test_queries, reference_answers=None, k=5, alpha=0.5):
    """
    评估融合检索与其他方法的比较。
    
    参数:
        pdf_path (str): PDF 文件的路径
        test_queries (List[str]): 测试查询列表
        reference_answers (List[str], optional): 参考答案
        k (int): 要检索的文档数量
        alpha (float): 融合检索中向量分数的权重
        
    返回:
        Dict: 评估结果
    """
    print("=== 评估融合检索 ===\n")
    
    # 处理文档以提取文本、创建块并构建向量和 BM25 索引
    chunks, vector_store, bm25_index = process_document(pdf_path)
    
    # 初始化一个列表来存储每个查询的结果
    results = []
    
    # 遍历每个测试查询
    for i, query in enumerate(test_queries):
        print(f"\n\n=== 评估查询 {i+1}/{len(test_queries)} ===")
        print(f"查询：{query}")
        
        # 如果可用，获取参考答案
        reference = None
        if reference_answers and i < len(reference_answers):
            reference = reference_answers[i]
        
        # 比较当前查询的检索方法
        comparison = compare_retrieval_methods(
            query, 
            chunks, 
            vector_store, 
            bm25_index, 
            k=k, 
            alpha=alpha,
            reference_answer=reference
        )
        
        # 将比较结果追加到结果列表中
        results.append(comparison)
        
        # 打印不同检索方法的响应
        print("\n=== 基于向量的响应 ===")
        print(comparison["vector_result"]["response"])
        
        print("\n=== BM25 响应 ===")
        print(comparison["bm25_result"]["response"])
        
        print("\n=== 融合响应 ===")
        print(comparison["fusion_result"]["response"])
        
        print("\n=== 比较 ===")
        print(comparison["comparison"])
    
    # 生成融合检索性能的总体分析
    overall_analysis = generate_overall_analysis(results)
    
    # 返回结果和总体分析
    return {
        "results": results,
        "overall_analysis": overall_analysis
    }

In [19]:
def generate_overall_analysis(results):
    """
    生成融合检索的总体分析。
    
    参数:
        results (List[Dict]): 评估查询的结果
        
    返回:
        str: 总体分析
    """
    # 指导评估过程的系统提示词
    system_prompt = """你是评估信息检索系统的专家。
    基于多个测试查询，提供比较三种检索方法的总体分析：
    1. 基于向量的检索（语义相似性）
    2. BM25 关键词检索（关键词匹配）
    3. 融合检索（两者结合）

    重点关注：
    1. 每种方法表现最佳的查询类型
    2. 每种方法的总体优缺点
    3. 融合检索如何平衡权衡
    4. 何时使用每种方法的建议"""

    # 为每个查询创建评估摘要
    evaluations_summary = ""
    for i, result in enumerate(results):
        evaluations_summary += f"查询 {i+1}：{result['query']}\n"
        evaluations_summary += f"比较摘要：{result['comparison'][:200]}...\n\n"

    # 包含评估摘要的用户提示词
    user_prompt = f"""基于以下对 {len(results)} 个查询的不同检索方法评估，
    提供比较这三种方法的总体分析：

    {evaluations_summary}

    请提供基于向量、BM25 和融合检索方法的全面分析，
    突出融合检索何时以及为什么比单独方法提供优势。"""

    # 使用 meta-llama/Llama-3.2-3B-Instruct 生成总体分析
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-3B-Instruct",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    
    # 返回生成的分析内容
    return response.choices[0].message.content

## 评估融合检索

In [20]:
# 要处理的 AI 信息文档的路径
pdf_path = "data/AI_Information.pdf"

# 定义涵盖不同类型查询的测试查询来评估融合检索
test_queries = [
    "transformer 模型在自然语言处理中的主要应用是什么？",
]

# 用于更彻底评估和比较结果的参考答案
reference_answers = [
    "Transformer 模型在自然语言处理中的主要应用包括机器翻译、文本生成、情感分析、文本分类、语言建模、问答系统和文本摘要。",
]

# 运行评估
evaluation_results = evaluate_fusion_retrieval(
    pdf_path=pdf_path,
    test_queries=test_queries,
    reference_answers=reference_answers,
    k=5,  # 检索前 5 个文档
    alpha=0.5  # 向量和 BM25 分数的平衡权重
)

# 打印总体分析
print("\n=== 总体分析 ===\n")
print(evaluation_results["overall_analysis"])

=== 评估融合检索 ===

从 data/AI_Information.pdf 提取文本...
创建了 42 个文本块
为块创建嵌入向量...
向向量存储添加了 42 个项目
创建了包含 42 个文档的 BM25 索引


=== 评估查询 1/1 ===
查询：transformer 模型在自然语言处理中的主要应用是什么？

=== 比较查询的检索方法：transformer 模型在自然语言处理中的主要应用是什么？ ===


运行仅向量 RAG...

运行仅 BM25 RAG...

运行融合 RAG...
对查询执行融合检索：transformer 模型在自然语言处理中的主要应用是什么？
使用融合检索检索到 5 个文档

比较响应...

=== 基于向量的响应 ===
提供的上下文没有明确提到 transformer 模型在自然语言处理中的主要应用。然而，上下文确实提到了生成式对抗网络（GANs）和 transformers 是生成式 AI 模型的例子，可以创建原创内容，包括图像、文本和音乐。

基于一般知识，transformer 模型在自然语言处理中广泛用于以下任务：

1. 机器翻译
2. 文本生成
3. 情感分析
4. 文本分类
5. 语言建模

这些模型在许多 NLP 任务中取得了最先进的结果，并已成为许多应用的热门选择。

如果您正在寻找有关 transformer 模型在 NLP 中应用的更具体信息，我可以尝试提供更多一般信息或为您指出更多资源的方向。

=== BM25 响应 ===
提供的上下文没有明确提到 transformer 模型或它们在自然语言处理中的应用。上下文涵盖了深度学习、卷积神经网络、循环神经网络、自然语言处理和机器学习等各种主题，但没有具体讨论 transformer 模型。

如果您正在寻找有关 transformer 模型的信息，我可以提供有关此主题的一般信息。Transformer 模型是一种神经网络架构，在机器翻译、文本生成和语言理解等自然语言处理任务中获得了广泛应用。它们在处理序列数据中的长程依赖关系方面特别有效，并已在许多 NLP 应用中被广泛采用。然而，这些信息在提供的上下文中并不存在。

=== 融合响应 ===
提供的上下文没有明确提到 transformer 模型在自然语言处理中的主要应用。然而，